# Matrix Factorization

In [4]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.01, Xinit = None, Winit = None, 
                 learning_rate = 0.01, max_iter = 1000, print_every = 100):
        self.Y_raw_data = Y_data
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        if Xinit is None: 
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: 
            self.W = Winit
            
        #self.all_users = self.Y_data[:,0] # all users (may be duplicated)
        self.n_ratings = Y_data.shape[0]
#         self.all_items = 

    def normalize_Y(self):
        users = self.Y_raw_data[:, 0] # all users - first col of the Y_data
        self.Y_data = self.Y_raw_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in xrange(self.n_users):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, 1] 
            # and the corresponding ratings 
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data[ids, 2] = ratings - self.mu[n]
    
    def loss(self):
        L = 0 
        for i in xrange(self.Y_data.shape[0]):
            # user, item, rating
            n, m, rate = int(self.Y_data[i, 0]), int(self.Y_data[i, 1]), self.Y_data[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
        # regularization, don't ever forget this 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # we need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data[:,0] == user_id)[0] 
        item_ids = self.Y_data[ids, 1].astype(np.int32) # index starts from 0 
        ratings = self.Y_data[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_items(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data[:,1] == item_id)[0] 
        user_ids = self.Y_data[ids, 0].astype(np.int32)
        ratings = self.Y_data[ids, 2]
        return (user_ids, ratings)
        
    def updateX(self):
        for m in xrange(self.n_items):
            user_ids, ratings = self.get_users_who_rate_items(m)
            Wm = self.W[:, user_ids]
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
    def updateW(self):
        for n in xrange(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            grad_wn = -Xn.T.dot(ratings- Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
    
    def fit(self):
        self.normalize_Y()
        for it in xrange(self.max_iter):
            self.updateX()
            self.updateW()
            if it % self.print_every == 0:
                print 'iter =', it, ', loss =', self.loss()
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        return self.X[i, :].dot(self.W[:, u]) + self.mu[u]
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in xrange(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
        
# data file 
# r_cols = ['user_id', 'item_id', 'rating']
# ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
# Y_data = ratings.as_matrix()


# rs = MF(Y_data, K = 2, max_iter = 10000, print_every = 1000)
# print rs.loss()
# print rs.get_items_rated_by_user(0)
# print rs.get_users_who_rate_items(0)
# rs.fit()
# rs.pred(6, 1)

# Áp dụng lên MovieLens 100k

In [5]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [47]:
rs = MF(rate_train, K = 500, lam = 1, print_every = 10, learning_rate = 0.75, max_iter = 100)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in xrange(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print 'Item-item CF, RMSE =', RMSE

iter = 0 , loss = 201.428782328
iter = 10 , loss = 0.2531996764
iter = 20 , loss = 0.253008722717
iter = 30 , loss = 0.253008722535
iter = 40 , loss = 0.253008722535
iter = 50 , loss = 0.253008722535
iter = 60 , loss = 0.253008722535
iter = 70 , loss = 0.253008722535
iter = 80 , loss = 0.253008722535
iter = 90 , loss = 0.253008722535
Item-item CF, RMSE = 1.06037990564


In [40]:
print rs.K

2


In [35]:
rs = MF(rate_train, K = 100, lam = 0.01, print_every = 50, learning_rate = 1, max_iter = 2, Xinit = rs.X, 
       Winit = rs.W)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in xrange(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print 'Item-item CF, RMSE =', RMSE

iter = 0 , loss = 0.25313267107
Item-item CF, RMSE = 1.06037990574


In [12]:
print rs.pred(1, 1)

6.68991073761


In [102]:
rs2 = MF(rate_train, K = 10, lam = 0.01, print_every = 20, learning_rate = 0.001, max_iter = 200)
rs2.fit()

iter = 0 , loss = 834968.255442
iter = 20 , loss = 63485.6626282
iter = 40 , loss = 42930.3071275
iter = 60 , loss = 37754.5380156
iter = 80 , loss = 35325.5213646
iter = 100 , loss = 33808.7359909
iter = 120 , loss = 32706.0820541
iter = 140 , loss = 31837.7213261
iter = 160 , loss = 31122.6640069
iter = 180 , loss = 30516.7608351


In [118]:
rs3 = MF(rate_train, K = 20, lam = 0.01, print_every = 20, \
         learning_rate = 0.1, max_iter = 500)
rs3.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in xrange(n_tests):
    pred = rs3.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print 'Item-item CF, RMSE =', RMSE

iter = 0 , loss = 18.2010515741
iter = 20 , loss = 17.2590357114
iter = 40 , loss = 16.4078277213
iter = 60 , loss = 15.6376045135
iter = 80 , loss = 14.9397357522
iter = 100 , loss = 14.3066209185
iter = 120 , loss = 13.7315513198
iter = 140 , loss = 13.2085928015
iter = 160 , loss = 12.7324857114
iter = 180 , loss = 12.2985592988
iter = 200 , loss = 11.9026582371
iter = 220 , loss = 11.5410793644
iter = 240 , loss = 11.2105170678
iter = 260 , loss = 10.9080160016
iter = 280 , loss = 10.6309300501
iter = 300 , loss = 10.3768866208
iter = 320 , loss = 10.1437555037
iter = 340 , loss = 9.92962165041
iter = 360 , loss = 9.73276132892
iter = 380 , loss = 9.55162119195
iter = 400 , loss = 9.38479986624
iter = 420 , loss = 9.23103172866
iter = 440 , loss = 9.08917258323
iter = 460 , loss = 8.95818699414
iter = 480 , loss = 8.83713706458
Item-item CF, RMSE = 4.05810732794


In [129]:
rs4 = MF(rate_train, K = 2, lam = 0.01, print_every = 20, \
         learning_rate = 0.5, max_iter = 500)
rs4.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in xrange(n_tests):
    pred = rs4.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print 'Item-item CF, RMSE =', RMSE

AttributeError: 'MF' object has no attribute 'Y_data'